In [ ]:
import warnings
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['M5'])
Query.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src[['A','Z']]                      = Query[['a','z']]
    Src[['Symbol','TF']]                = Query[['symbol','tf']]
    
    Src['Datetime']                     = Query['datetime']
    Src['Date']                         = Query['datetime'].dt.date
    Src['Time']                         = Query['datetime'].dt.time
    Src['ID T']                         = Src.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)['Time'].rank()

    Src[['Ticks','Volume']]             = Query[['tick_volume','real_volume']]
    Src[['Open','High','Low','Close']]  = Query[['open','high','low','close']]
    Src[['Price']]                      = Query[['close']]

    # Src['Change']       = (Src['Close'] - Src['Open'])
    # Src['Hilo']         = (Src['High']  - Src['Low'])

    # Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if x >= 0 else nan)
    # Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if x <  0 else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)
    
    Calc['Day Open']        = Calc_by['High'] .transform('first')
    Calc['Day High']        = Calc_by['High'] .transform('max')
    Calc['Day Low']         = Calc_by['Low']  .transform('min')
    Calc['Day Close']       = Calc_by['High'] .transform('last')

    Calc['Day Hilo']        = Calc['Day High']  - Calc['Day Low'] 
    Calc['Day Change']      = Calc['Day Close'] - Calc['Day Open'] 
    Calc['Day Chg Abs']     = Calc['Day Change'].abs()


    Calc['ID Cumax']        = Calc_by['High'] .expanding().max()    .reset_index(drop=1)
    Calc['ID Cumin']        = Calc_by['Low']  .expanding().min()    .reset_index(drop=1)
    Calc['ID CumHL']        = Calc['ID Cumax'] - Calc['ID Cumin']

    Calc['ID did Max']      = (Calc['ID Cumax'] >= Calc['Day High'])
    Calc['ID did Min']      = (Calc['ID Cumin'] <= Calc['Day Low'])

    Calc['ID did Max +M60'] = Calc_by['ID did Max'] .shift(-12+1)
    Calc['ID did Min +M60'] = Calc_by['ID did Min'] .shift(-12+1)


    Calc['Fwd Hilo']        = Calc['Day Hilo']  - Calc['ID CumHL']
    Calc['Fwd Change']      = Calc['Day Close'] - Calc['Close']
    Calc['Fwd Chg Abs']     = Calc['Fwd Change'].abs()


    Calc['Variat M5']       = (Calc['Close'] - Calc_by['Open'].shift( +1 -1))
    Calc['Variat M15']      = (Calc['Close'] - Calc_by['Open'].shift( +3 -1))
    Calc['Variat M60']      = (Calc['Close'] - Calc_by['Open'].shift(+12 -1))

    Calc['Return +M5']      = (Calc_by['Close'].shift( -1 +1) - Calc['Open'])
    Calc['Return +M15']     = (Calc_by['Close'].shift( -3 +1) - Calc['Open'])
    Calc['Return +M60']     = (Calc_by['Close'].shift(-12 +1) - Calc['Open'])


    Calc['Var Sgn M5']      = Calc['Variat M5']   .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Var Sgn M15']     = Calc['Variat M15']  .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Var Sgn M60']     = Calc['Variat M60']  .apply(lambda x: -1 if (x < 0) else +1) 

    Calc['Rtn Sgn +M5']     = Calc['Return +M5']  .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Rtn Sgn +M15']    = Calc['Return +M15'] .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Rtn Sgn +M60']    = Calc['Return +M60'] .apply(lambda x: -1 if (x < 0) else +1) 


    Calc['Var Comb Bull']   = Calc.loc[:, 'Var Sgn M5':'Var Sgn M60'].eq(+1).all(axis=1)
    Calc['Var Comb Bear']   = Calc.loc[:, 'Var Sgn M5':'Var Sgn M60'].eq(-1).all(axis=1)

    Calc['Rtn Comb Bull']   = Calc.loc[:, 'Rtn Sgn +M5':'Rtn Sgn +M60'].eq(+1).all(axis=1)
    Calc['Rtn Comb Bear']   = Calc.loc[:, 'Rtn Sgn +M5':'Rtn Sgn +M60'].eq(-1).all(axis=1)
    return Calc

Calc = CALCULATIONS(Src)
Calc

# Stats - Counter

In [33]:
pd.Series({
    'Rtn Comb Bull': np.round(Calc[Calc['ID T'].eq(1)]['Rtn Comb Bull'].mean() *100, 1),
    'Rtn Comb Bear': np.round(Calc[Calc['ID T'].eq(1)]['Rtn Comb Bear'].mean() *100, 1),
})

Rtn Comb Bull    30.8
Rtn Comb Bear    32.2
dtype: float64

# Stats - Hilos

In [34]:
pd.DataFrame([
    { 'Cond':'Rtn Comb Bull', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bull']][['ID did Max +M60','ID did Min +M60']].mean().mul(100).apply(pd.to_numeric).round(1) }, 
    { 'Cond':'Rtn Comb Bear', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bear']][['ID did Max +M60','ID did Min +M60']].mean().mul(100).apply(pd.to_numeric).round(1) }, 
])

,Cond,ID did Max +M60,ID did Min +M60
0,Rtn Comb Bull,16.5,43.4
1,Rtn Comb Bear,42.6,14.1


# Stats - Daily Amplitude

In [35]:
pd.DataFrame([
    { 'Cond':'Rtn Comb Bull', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bull']]['Day Hilo'].describe().round(0).astype(int) }, 
    { 'Cond':'Rtn Comb Bear', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bear']]['Day Hilo'].describe().round(0).astype(int) }, 
])

,Cond,count,mean,std,min,25%,50%,75%,max
0,Rtn Comb Bull,279,1923,869,610,1390,1680,2240,7400
1,Rtn Comb Bear,291,1899,798,660,1360,1715,2278,4625


In [36]:
pd.DataFrame([
    { 'Cond':'Rtn Comb Bull', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bull']]['Day Chg Abs'].describe().round(0).astype(int) }, 
    { 'Cond':'Rtn Comb Bear', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bear']]['Day Chg Abs'].describe().round(0).astype(int) }, 
])

,Cond,count,mean,std,min,25%,50%,75%,max
0,Rtn Comb Bull,279,962,820,5,360,760,1322,4525
1,Rtn Comb Bear,291,1045,829,5,422,875,1502,4085


In [37]:
pd.DataFrame([
    { 'Cond':'Rtn Comb Bull', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bull']]['Day Change'].describe().round(0).astype(int) }, 
    { 'Cond':'Rtn Comb Bear', **Calc[Calc['ID T'].eq(1) & Calc['Rtn Comb Bear']]['Day Change'].describe().round(0).astype(int) }, 
])

,Cond,count,mean,std,min,25%,50%,75%,max
0,Rtn Comb Bull,279,286,1233,-3295,-432,245,992,4525
1,Rtn Comb Bear,291,-546,1219,-4085,-1168,-585,120,3575


# Stats - Points from first combo

In [38]:
pd.DataFrame([
    { 'Cond':'Var Comb Bull', **Calc[Calc['ID T'].eq(12) & Calc['Var Comb Bull']]['Fwd Change'].describe().round(0).astype(int) }, 
    { 'Cond':'Var Comb Bear', **Calc[Calc['ID T'].eq(12) & Calc['Var Comb Bear']]['Fwd Change'].describe().round(0).astype(int) }, 
])

,Cond,count,mean,std,min,25%,50%,75%,max
0,Var Comb Bull,214,124,1167,-3300,-619,212,819,4100
1,Var Comb Bear,192,-23,1085,-2390,-726,-65,609,3195


In [39]:
pd.DataFrame([
    { 'Cond':'Var Comb Bull', **Calc[Calc['ID T'].eq(12) & Calc['Var Comb Bull']]['Fwd Chg Abs'].describe().round(0).astype(int) }, 
    { 'Cond':'Var Comb Bear', **Calc[Calc['ID T'].eq(12) & Calc['Var Comb Bear']]['Fwd Chg Abs'].describe().round(0).astype(int) }, 
])

,Cond,count,mean,std,min,25%,50%,75%,max
0,Var Comb Bull,214,910,738,5,360,710,1269,4100
1,Var Comb Bear,192,848,674,25,325,710,1159,3195


In [40]:
pd.DataFrame([
    { 'Cond':'Var Comb Bull', **Calc[Calc['ID T'].eq(12) & Calc['Var Comb Bull']]['Fwd Hilo'].describe().round(0).astype(int) }, 
    { 'Cond':'Var Comb Bear', **Calc[Calc['ID T'].eq(12) & Calc['Var Comb Bear']]['Fwd Hilo'].describe().round(0).astype(int) }, 
])

,Cond,count,mean,std,min,25%,50%,75%,max
0,Var Comb Bull,214,1142,707,55,675,995,1459,4180
1,Var Comb Bear,192,1139,651,35,688,1005,1446,3765


# Snippets

In [42]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc_by = Calc.groupby(['Symbol','TF','Date'], sort=0, group_keys=0)
    
    Calc['Day Open']        = Calc_by['High'] .transform('first')
    Calc['Day High']        = Calc_by['High'] .transform('max')
    Calc['Day Low']         = Calc_by['Low']  .transform('min')
    Calc['Day Close']       = Calc_by['High'] .transform('last')

    Calc['Day Hilo']        = Calc['Day High']  - Calc['Day Low'] 
    Calc['Day Change']      = Calc['Day Close'] - Calc['Day Open'] 
    Calc['Day Chg Abs']     = Calc['Day Change'].abs()


    Calc['ID Cumax']        = Calc_by['High'] .expanding().max()    .reset_index(drop=1)
    Calc['ID Cumin']        = Calc_by['Low']  .expanding().min()    .reset_index(drop=1)
    Calc['ID CumHL']        = Calc['ID Cumax'] - Calc['ID Cumin']

    Calc['ID did Max']      = (Calc['ID Cumax'] >= Calc['Day High'])
    Calc['ID did Min']      = (Calc['ID Cumin'] <= Calc['Day Low'])

    Calc['ID did Max +M60'] = Calc_by['ID did Max'] .shift(-12+1)
    Calc['ID did Min +M60'] = Calc_by['ID did Min'] .shift(-12+1)


    Calc['Fwd Hilo']        = Calc['Day Hilo']  - Calc['ID CumHL']
    Calc['Fwd Change']      = Calc['Day Close'] - Calc['Close']
    Calc['Fwd Chg Abs']     = Calc['Fwd Change'].abs()
    
	...

TabError: inconsistent use of tabs and spaces in indentation (<string>, line 30)

In [ ]:
def CALCULATIONS(Src):
	...
    
    Calc['Variat M5']       = (Calc['Close'] - Calc_by['Open'].shift( +1 -1))
    Calc['Variat M15']      = (Calc['Close'] - Calc_by['Open'].shift( +3 -1))
    Calc['Variat M60']      = (Calc['Close'] - Calc_by['Open'].shift(+12 -1))

    Calc['Return +M5']      = (Calc_by['Close'].shift( -1 +1) - Calc['Open'])
    Calc['Return +M15']     = (Calc_by['Close'].shift( -3 +1) - Calc['Open'])
    Calc['Return +M60']     = (Calc_by['Close'].shift(-12 +1) - Calc['Open'])


    Calc['Var Sgn M5']      = Calc['Variat M5']   .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Var Sgn M15']     = Calc['Variat M15']  .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Var Sgn M60']     = Calc['Variat M60']  .apply(lambda x: -1 if (x < 0) else +1) 

    Calc['Rtn Sgn +M5']     = Calc['Return +M5']  .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Rtn Sgn +M15']    = Calc['Return +M15'] .apply(lambda x: -1 if (x < 0) else +1) 
    Calc['Rtn Sgn +M60']    = Calc['Return +M60'] .apply(lambda x: -1 if (x < 0) else +1) 


    Calc['Var Comb Bull']   = Calc.loc[:, 'Var Sgn M5':'Var Sgn M60'].eq(+1).all(axis=1)
    Calc['Var Comb Bear']   = Calc.loc[:, 'Var Sgn M5':'Var Sgn M60'].eq(-1).all(axis=1)

    Calc['Rtn Comb Bull']   = Calc.loc[:, 'Rtn Sgn +M5':'Rtn Sgn +M60'].eq(+1).all(axis=1)
    Calc['Rtn Comb Bear']   = Calc.loc[:, 'Rtn Sgn +M5':'Rtn Sgn +M60'].eq(-1).all(axis=1)
    return Calc